In this notebook, we'll explore few-shot learning with GPT-2.  While GPT-2 is a less expressive model than GPT-3 (and hence not as a good of a few shot learner), it can fit within the memory and processing constraints of laptops while also being openly available.  Can you create a new classification task and design prompts to differentiate between the classes within it?

In [ ]:
import torch
from torch.nn import functional as F

In [ ]:
from transformers import pipeline

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('gpt2')
model = AutoModelForCausalLM.from_pretrained('gpt2')

In [ ]:
def classify_with_prompt(prompt, labels):
    inputs = tokenizer.encode(prompt, return_tensors='pt')
    completion_layer = model(inputs).logits[:, -1, :]
    probabilities = F.softmax(completion_layer, dim=-1)[0]
    pred_idx=torch.argmax(probabilities)
    pred_token = tokenizer.decode(pred_idx.tolist())

    label_ids=[]
    for label in labels:
        token_ids=tokenizer.encode(label)
        # token labels (e.g., Spanish, English) must be 1 token in length
        assert len(token_ids) == 1
        label_ids.append(token_ids[0])
        
    sorted_args=list(torch.argsort(probabilities[label_ids], descending=True))
    for arg in sorted_args: 
            print("%.6f\t%s" % (probabilities[label_ids[arg]], labels[arg]))
    
    print("\nCompletion with highest probability:\n")
    print(prompt + pred_token)

In [ ]:
prompt = """X: I love this movie
Y: positive

X: I hate the movie
Y: negative

X: I kind of like the movie
Y: positive

X: This is one of the best movies I've ever seen
Y:"""

classify_with_prompt(prompt, ["positive", "negative"])

In [ ]:
prompt = """X: Vampires take over the planet during an eclipse
Y: Hor

X: Two friends switch bodies and live each other's lives
Y: Com

X: John turns into a werewolf during a full moon
Y: Hor

X: John is a werewolf who plays basketball
Y: Com

X: A court sentences George to be Jerry's butler
Y: Com

X: A virus outbreak turns everyone into zombies
Y:"""

classify_with_prompt(prompt, ["Hor", "Com"])

In [ ]:
prompt = """Q: This is a text
A: English

Q: Nel mezzo del cammin' di nostra vita
A: Italian

Q: Je ne sais pas
A:"""

classify_with_prompt(prompt, ["English", "Italian", "French", "Spanish", "Japanese"])

**Q1**.  Your job is to create a new classification task using prompt design (as in the examples above).  You are free to consider binary classification or multiclass classifaction; keep in mind that you have ~1000 tokens to use as a prompt for GPT-2, so be sure to provide enough answered prompts for each class.  (Note it is not a requirement that your model performs *well* (we want to assess what is -- and isn't -- learnable) but give it every opportunity to do so.  Create 5 test examples to assess whether GPT-2 is able to recognize the class given your fixed prompt.  To take the language ID task above, one test example corresponds to one prediction you make for the same set of answered prompts; the following constitutes two test examples for that task:

1.)

```
prompt = """Q: This is a text
A: English

Q: Nel mezzo del cammin' di nostra vita
A: Italian

Q: Je ne sais pas
A:"""
```

2.)

``` 
prompt = """Q: This is a text
A: English

Q: Nel mezzo del cammin' di nostra vita
A: Italian

Q: Non lo so
A:"""
```